In [ ]:
%install pyrefdata 0.4.0
#%pip install holidays

In [ ]:
# from typing import List, Optional, Union
# import pandas as pd
# import pyrefdata
# import bqapi


# def format_datestring(date: Union[str, pd.Timestamp]) -> str:
#     """Formats the date as YYYYMMDD string."""
#     return pd.Timestamp(date).strftime("%Y%m%d")


# def get_historical_px_last(
#     securities: List[str],
#     start_date: Union[str, pd.Timestamp],
#     end_date: Union[str, pd.Timestamp],
#     currency: str = "BRL",
#     frequency: str = "monthly",  # 'daily' or 'monthly'
#     **kwargs,
# ) -> pd.DataFrame:
#     """
#     Get historical PX_LAST prices for a list of securities using pyrefdata.

#     Parameters
#     ----------
#     securities : list of str
#         Bloomberg tickers, e.g., ['ED345652@ande Corp']
#     start_date : str or pd.Timestamp
#         Start date, e.g., '2005-03-10'
#     end_date : str or pd.Timestamp
#         End date, e.g., '2012-01-01'
#     currency : str
#         Currency override.
#     frequency : str
#         'daily' or 'monthly'.
#     kwargs : dict
#         Additional arguments to pass to bqapi.Session

#     Returns
#     -------
#     pd.DataFrame
#         Date-indexed DataFrame with securities as columns.
#     """
#     start = format_datestring(start_date)
#     end = format_datestring(end_date)

#     with bqapi.Session(**kwargs) as session:
#         df = pyrefdata.get_historical_data(
#             session,
#             securities,
#             fields=["PX_LAST"],
#             start_date=start,
#             end_date=end,
#             currency=currency,
#             non_trading_day_fill_option="all_calendar_days",
#             override_option="close",
#             ignore_errors=True,
#         ).reset_index()

#     # Pivot the result to get one column per security
#     df = df.pivot(index="date", columns="Security", values="PX_LAST").ffill()

#     # Ensure the index is datetime before resampling
#     df.index = pd.to_datetime(df.index)

#     # Monthly resample if requested
#     if frequency == "monthly":
#         df = df.resample("M").last()

#     df.index.name = "DATE"
#     return df


In [ ]:
# securities = ['ED345652@ande Corp']



# start_date = '2005-03-10'
# end_date = '2012-01-01'

# df = get_historical_px_last(
#     securities,
#     start_date=start_date,
#     end_date=end_date,
#     currency="BRL",
#     frequency="monthly"  # or 'daily'
# )

# print(df)

In [ ]:
from typing import List, Union, Dict, Optional
import pandas as pd
import pyrefdata
import bqapi
import os
from datetime import date

def format_datestring(date: Union[str, pd.Timestamp]) -> str:
    return pd.Timestamp(date).strftime("%Y%m%d")


def fetch_historical_data_for_sources(
    base_ticker: str,
    pricing_sources: List[str],
    start_date: Union[str, pd.Timestamp],
    end_date: Union[str, pd.Timestamp],
    currency: str = "BRL",
    frequency: str = "monthly",
    fallback_years: int = 10,
    **kwargs,
) -> Dict[str, Union[Dict[str, pd.DataFrame], str]]:
    """
    Fetch prices from all sources. Evaluates best source with most data. Includes correct NaN handling.
    """
    start_fmt = format_datestring(start_date)
    end_fmt = format_datestring(end_date)
    all_data: Dict[str, pd.DataFrame] = {}
    diagnostics: Dict[str, str] = {}

    for source in pricing_sources:
        ticker = f"{base_ticker}@{source.lower()} Corp"
        try:
            with bqapi.Session(**kwargs) as session:
                df = pyrefdata.get_historical_data(
                    session,
                    [ticker],
                    fields=["PX_LAST"],
                    start_date=start_fmt,
                    end_date=end_fmt,
                    currency=currency,
                    non_trading_day_fill_option="all_calendar_days",
                    override_option="close",
                    ignore_errors=True,
                ).reset_index()

                if df.empty:
                    diagnostics[source] = "Empty in original window"
                    # Try fallback to recent years
                    df = pyrefdata.get_historical_data(
                        session,
                        [ticker],
                        fields=["PX_LAST"],
                        start_date=format_datestring(pd.Timestamp.today() - pd.DateOffset(years=fallback_years)),
                        end_date=format_datestring(pd.Timestamp.today()),
                        currency=currency,
                        non_trading_day_fill_option="all_calendar_days",
                        override_option="close",
                        ignore_errors=True,
                    ).reset_index()

                    if df.empty:
                        diagnostics[source] += " + fallback failed"
                        continue
                    else:
                        diagnostics[source] += " + fallback success"

                df = df.pivot(index="date", columns="Security", values="PX_LAST")
                df.index = pd.to_datetime(df.index)

                if frequency == "monthly":
                    df = df.resample("M").last()

                # Fill internal gaps only
                df = df.ffill()

                # Truncate after last real price
                last_valid_index = df.last_valid_index()
                if last_valid_index is not None:
                    df = df.loc[:last_valid_index]

                df.index.name = "DATE"
                all_data[source] = df
                diagnostics[source] = diagnostics.get(source, "Success")

        except Exception as e:
            diagnostics[source] = f"Exception: {str(e)}"
            continue

    contribution_counts = {
        source: df.notna().sum().sum() for source, df in all_data.items()
    }

    best_source = (
        max(contribution_counts.items(), key=lambda x: x[1])[0]
        if contribution_counts else None
    )

    return {
        "data": all_data,
        "best_source": best_source,
        "diagnostics": diagnostics,
    }

def build_actual_observation_table(
    tickers: List[str],
    pricing_sources: List[str],
    start_date: Union[str, pd.Timestamp],
    end_date: Union[str, pd.Timestamp],
    currency: str = "BRL",
    fallback_years: int = 10,
    **kwargs,
) -> pd.DataFrame:
    """
    Returns a DataFrame:
    - Rows: All calendar dates in range [start_date, end_date]
    - Columns: <ticker>@<best_source>
    - Values: Actual PX_LAST observations only, no forward-fill
    """
    combined_df = pd.DataFrame()
    diagnostics = {}

    # Define full date range index
    full_index = pd.date_range(start=start_date, end=end_date, freq='D')

    for ticker in tickers:
        print(f"Processing {ticker}...")

        result = fetch_historical_data_for_sources(
            ticker,
            pricing_sources,
            start_date,
            end_date,
            frequency="daily",  # Get raw data
            currency=currency,
            fallback_years=fallback_years,
            **kwargs,
        )

        best_source = result['best_source']
        diagnostics[ticker] = best_source

        if not best_source:
            print(f"  ❌ No data for {ticker}")
            continue

        df = result['data'][best_source].copy()
        df.index = pd.to_datetime(df.index)
        df = df.reindex(full_index)  # Align to full date range
        df.columns = [f"{ticker}@{best_source}"]

        combined_df = combined_df.join(df, how="outer") if not combined_df.empty else df

    print("\n✅ Pricing Sources Used:")
    for k, v in diagnostics.items():
        print(f"  {k}: {v or '❌ No data'}")

    combined_df.index.name = "DATE"
    return combined_df


In [ ]:
# xls = pd.ExcelFile("universo_brazil_deb.xlsx")
# df = xls.parse(sheet_name='Sheet1')
# tickers = df["Ticker"].dropna().unique().tolist()

In [ ]:
xls = pd.ExcelFile("pcs_debentures.xlsx")
df = xls.parse(sheet_name='Sheet1')
pricing_sources = df["pricing_source"].dropna().unique().tolist()

In [ ]:
#pricing_sources = ['bval', 'psan', 'brad', 'cmdb', 'anbe', 'ande', 'icor', 'miae', 'xpcp']
#pricing_sources = pricing_sources[0:5]
#tickers = ['ED345652','TEQU11','MNRV16']
#tickers = tickers[0:3]


xls = pd.ExcelFile("parte_4.xlsx")
df = xls.parse(sheet_name='Sheet1')
tickers = df["Ticker"].dropna().unique().tolist()

df = build_actual_observation_table(
    tickers,
    pricing_sources,
    start_date="1989-10-01",
    end_date="2025-08-04",
)
   

df


df.to_excel("resultado_parte_4.xlsx")
print(f"✅ File 4 saved")

In [ ]:


# Use today's date in format YYYY.MM.DD
today = date.today().strftime('%Y.%m.%d')

# Set output path
home_directory = os.path.expanduser('~')
export_filename = f"{home_directory}/Downloads/waterfallbondpriced-{today}.xlsx"

# Assuming your DataFrame is called df_fixed
with pd.ExcelWriter(export_filename, date_format='yyyy/mm/dd',
                    datetime_format='yyyy/mm/dd') as writer:
    df.to_excel(writer)

print(f"✅ File saved to: {export_filename}")


In [ ]:
df.to_excel("resultado_parte_6.xlsx")

In [ ]:
#quebrar a lista de tickers em 20 pedaços pois é muito grande pra processar de uma vez


import pandas as pd

df = pd.read_excel('universo_brazil_deb.xlsx')  

coluna = df['Ticker']

partes = []
tamanho_parte = len(coluna) // 20 + (len(coluna) % 20 > 0)

for i in range(0, len(coluna), tamanho_parte):
    partes.append(coluna.iloc[i:i+tamanho_parte])

for i, parte in enumerate(partes):
    parte.to_excel(f'parte_{i+1}.xlsx', index=False)

In [ ]:
df

In [2]:
import pandas as pd
import numpy as np
import glob, os, re, gc
from openpyxl import Workbook

# =========================
# CONFIG
# =========================
PATTERN = "resultado_parte_*.xlsx"   # ou "parte_*.xlsx"
OUTFILE = "resultado_unico.xlsx"
DATE_CANDIDATES = ["DATE", "Date", "date", "DATA"]
EXCEL_MAX_COLS = 16384  # limite do Excel (colunas)
PRINT_EVERY_FILE = 1    # print de progresso a cada N arquivos
PRINT_EVERY_ROWS = 5000 # print de progresso ao escrever linhas no Excel

# =========================
# HELPERS
# =========================
def numeric_key(path):
    m = re.search(r'(\d+)', os.path.basename(path))
    return int(m.group(1)) if m else 0

def read_part(path):
    """Lê uma parte, normaliza a coluna DATE como índice datetime e converte valores para float32."""
    df = pd.read_excel(path)
    # encontrar/normalizar DATE
    date_col = next((c for c in DATE_CANDIDATES if c in df.columns), df.columns[0])
    if date_col != "DATE":
        df = df.rename(columns={date_col: "DATE"})
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")
    df = df.dropna(subset=["DATE"]).set_index("DATE")

    # converter colunas não-numéricas para numérico (coerce -> NaN), depois forçar float32
    non_num = [c for c in df.columns if not np.issubdtype(df[c].dtype, np.number)]
    if non_num:
        df[non_num] = df[non_num].apply(pd.to_numeric, errors="coerce")
    df = df.astype(np.float32)
    return df

# =========================
# MERGE (UNION DE TICKERS) POR DATE
# =========================
files = sorted(glob.glob(PATTERN), key=numeric_key)
if not files:
    raise FileNotFoundError(f"Nenhum arquivo encontrado para o padrão: {PATTERN}")

print(f"Found {len(files)} files. Merging into one sheet…")

master = None
for i, f in enumerate(files, 1):
    part = read_part(f)

    if master is None:
        master = part
    else:
        # se houver tickers em comum, priorize valores já existentes (non-NaN) no master
        overlap = master.columns.intersection(part.columns)
        if len(overlap) > 0:
            master[overlap] = master[overlap].combine_first(part[overlap])
            part = part.drop(columns=overlap)

        master = master.join(part, how="outer")

    if i % PRINT_EVERY_FILE == 0:
        print(f"  -> merged {os.path.basename(f)} | shape now {master.shape}")
    del part
    gc.collect()

# ordenar por data e por nome do ticker (opcional)
master = master.sort_index()
master = master.reindex(columns=sorted(master.columns, key=str))

# checar limite de colunas do Excel (DATE + tickers)
total_cols_for_excel = master.shape[1] + 1
if total_cols_for_excel > EXCEL_MAX_COLS:
    raise RuntimeError(
        f"Too many columns for Excel: {total_cols_for_excel} > {EXCEL_MAX_COLS}. "
        f"Reduza colunas ou salve em Parquet/CSV."
    )

print(
    f"\nFinal shape: {master.shape[0]} rows x {master.shape[1]} ticker columns "
    f"(+ DATE = {total_cols_for_excel} total Excel columns)"
)

# =========================
# SALVAR COM OPENPYXL (WRITE-ONLY STREAMING)
# =========================
wb = Workbook(write_only=True)
ws = wb.create_sheet("dados")

# remover a sheet padrão criada pelo Workbook()
try:
    wb.remove(wb["Sheet"])
except Exception:
    pass

# cabeçalho
header = ["DATE"] + master.columns.tolist()
ws.append(header)

# função rápida para converter NaN -> None (openpyxl não grava NaN)
def nan_to_none_row(row_tuple):
    arr = np.array(row_tuple, dtype=object)
    # pandas.isna lida com floats e None
    mask = pd.isna(arr)
    if mask.any():
        arr[mask] = None
    return arr.tolist()

# escrever linhas
for k, (dt, row) in enumerate(zip(master.index.to_pydatetime(),
                                  master.itertuples(index=False, name=None)), start=1):
    ws.append([dt] + nan_to_none_row(row))
    if k % PRINT_EVERY_ROWS == 0:
        print(f"  -> wrote {k} rows to Excel...")

wb.save(OUTFILE)
print(f"✅ Saved: {OUTFILE}")

# =========================
# ALTERNATIVA SIMPLES (se memória permitir):
# master.to_excel(OUTFILE, index_label='DATE')
# =========================


Found 20 files. Merging into one sheet…
  -> merged resultado_parte_1.xlsx | shape now (13092, 179)
  -> merged resultado_parte_2.xlsx | shape now (13092, 413)
  -> merged resultado_parte_3.xlsx | shape now (13092, 659)
  -> merged resultado_parte_4.xlsx | shape now (13092, 662)
  -> merged resultado_parte_5.xlsx | shape now (13092, 952)
  -> merged resultado_parte_6.xlsx | shape now (13092, 1243)
  -> merged resultado_parte_7.xlsx | shape now (13092, 1544)
  -> merged resultado_parte_8.xlsx | shape now (13092, 1692)
  -> merged resultado_parte_9.xlsx | shape now (13092, 1789)
  -> merged resultado_parte_10.xlsx | shape now (13092, 1892)
  -> merged resultado_parte_11.xlsx | shape now (13092, 1974)
  -> merged resultado_parte_12.xlsx | shape now (13092, 2075)
  -> merged resultado_parte_13.xlsx | shape now (13092, 2175)
  -> merged resultado_parte_14.xlsx | shape now (13092, 2314)
  -> merged resultado_parte_15.xlsx | shape now (13092, 2564)
  -> merged resultado_parte_16.xlsx | shape 